In [1]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import load_data, get_label_dict, get_word_labels

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

Using TensorFlow backend.


In [2]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data



In [3]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

Loading labels

Done

Loading words

Done



In [4]:
# Image preprocessing for the data
datagen = ImageDataGenerator(rotation_range=40, 
                             brightness_range=[0.6, 1.4], 
                             shear_range=0.2, 
                             zoom_range=0.2, 
                             horizontal_flip=True, 
                             vertical_flip=True, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=90., 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [5]:
model = AlphaNet()
model.name = "AlphaNet_v2"
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

In [6]:
# Checkpoint callback to only save best model
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             mode="auto", 
                             period=1)

cb_list = [checkpoint]

In [8]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=False, 
                epochs=300,
                model=model, 
                optim=optim,
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Model: "AlphaNet_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        

3125/3125 [==============================] - 201s 64ms/step - loss: 4.7823 - accuracy: 0.0474 - top3_accuracy: 0.1102 - top5_accuracy: 0.1589 - val_loss: 4.5416 - val_accuracy: 0.0588 - val_top3_accuracy: 0.1308 - val_top5_accuracy: 0.1828

Epoch 00010: val_accuracy improved from 0.04970 to 0.05880, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 11/300
3125/3125 [==============================] - 197s 63ms/step - loss: 4.7438 - accuracy: 0.0513 - top3_accuracy: 0.1185 - top5_accuracy: 0.1680 - val_loss: 4.5763 - val_accuracy: 0.0675 - val_top3_accuracy: 0.1447 - val_top5_accuracy: 0.2034

Epoch 00011: val_accuracy improved from 0.05880 to 0.06750, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 12/300
3125/3125 [==============================] - 199s 64ms/step - loss: 4.7088 - accuracy: 0.0547 - top3_accuracy: 0.1249 - top5_accuracy: 0.1762 - val_loss: 4.3047 - val_accuracy: 0.0734 - val_top3_accuracy: 0.1579 - val_top5_accuracy: 0.2206

Epoch 00012: val_accuracy impro

Epoch 34/300
3125/3125 [==============================] - 195s 62ms/step - loss: 4.3548 - accuracy: 0.0958 - top3_accuracy: 0.1991 - top5_accuracy: 0.2664 - val_loss: 3.9529 - val_accuracy: 0.1294 - val_top3_accuracy: 0.2546 - val_top5_accuracy: 0.3304

Epoch 00034: val_accuracy did not improve from 0.12970
Epoch 35/300
3125/3125 [==============================] - 193s 62ms/step - loss: 4.3480 - accuracy: 0.0971 - top3_accuracy: 0.1994 - top5_accuracy: 0.2679 - val_loss: 3.9125 - val_accuracy: 0.1302 - val_top3_accuracy: 0.2584 - val_top5_accuracy: 0.3379

Epoch 00035: val_accuracy improved from 0.12970 to 0.13020, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 36/300
3125/3125 [==============================] - 195s 62ms/step - loss: 4.3387 - accuracy: 0.0978 - top3_accuracy: 0.2011 - top5_accuracy: 0.2692 - val_loss: 3.9150 - val_accuracy: 0.1311 - val_top3_accuracy: 0.2481 - val_top5_accuracy: 0.3265

Epoch 00036: val_accuracy improved from 0.13020 to 0.13110, saving model 

KeyboardInterrupt: 

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model(model_path, custom_objects=custom_metrics)
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    img = Image.fromarray(X_test[index], "RGB").resize(size=(display_dim, display_dim))
    word_label = words[results[index]]
    display(img)
    print(word_label)

In [ ]:
# This saves an HTML version of the notebook for later viewing
os.system("%notebook 'Training'")
os.system("jupyter nbconvert --to html Training.ipynb")

html_path = model_prefix + ".html"
if path.exists(html_path):
    os.remove(html_path)
os.rename("Training.html", html_path)